In [1]:
import pyscf
import pyscf.tools

from spade_clustering import *

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


In [2]:
molecule = """
H 0.0 0.0 0.0
H 2.0 0.0 0.0
H 4.0 0.0 0.0
He 0.0 2.0 0.0
He 2.0 2.0 0.0
He 4.0 2.0 0.0
He 6.0 2.0 0.0
He 8.0 2.0 0.0
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   1, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  Cs


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
DIIS start cycle = 1
DIIS space = 8
SCF tol = 1e-08
SCF gradient tol = 1e-05
max. SCF cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 6  num. singly occ = 1
init E= -15.3068786812974
HOMO (A') = -0.174946216158305  LUMO (A') = -0.0514772431085871
cycle= 1 E= -15.6442542564128  delta_E= -0.337  |g|= 0.05  |ddm|= 0.768
HOMO (A') = -0.21441865770599  LUMO (A') = 0.00552451182760706
cycle= 2 E= -15.6461073296957  delta_E= -0.00185  |g|= 0.00752  |ddm|= 0.0726
HOMO (A') = -0.209411568433181  LUMO (A') = 0.00747490869695385
cycle= 3 E= -15.6461544034177  delta_E= -4.71e-05  |g|= 0.00116  |ddm|= 0.0134
HOMO (A') = -0.209064623291325  LUMO (A') = 0.00775725750823334
cycle= 4 E= -15.6461554

# Define Fragments by AOs

In [3]:
# Find AO's corresponding to atoms 
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] in (0, 1, 2, 3):
        if ao[2] in ("1s",):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] in (4, 5):
        if ao[2] in ("1s", ):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("1s", ):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[0, 5, 10, 15], [20, 25], [30]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [4]:
# Define projectors
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [5]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = spade_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact,Vact))

 Partition   40 orbitals into a total of    7 orbitals
            Index   Sing. Val. Space       
                0   0.92782623            2*
                1   0.84273729            0*
                2   0.83388766            0*
                3   0.82720440            0*
                4   0.82048000            0*
                5   0.81282917            1*
                6   0.71276005            0*
                7   0.17409277            2
                8   0.16456991            2
                9   0.15748413            2
               10   0.14786680            2
               11   0.14198651            2
               12   0.12123142            2
               13   0.00036667            0


# Split active space into fragments

In [6]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 0
for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = spade_partitioning((Cdocc, Csing, Cvirt), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace: ", init_fspace)
print(" clusters   : ", clusters)



 Fragment:  [0, 5, 10, 15]
 Partition   40 orbitals into a total of    4 orbitals
            Index   Sing. Val. Space       
                0   0.92202281            2*
                1   0.83007666            0*
                2   0.80741097            1*
                3   0.71836264            0*
                4   0.17133587            2
                5   0.15202631            2
                6   0.12384227            2
                7   0.08582984            0
                8   0.04379145            0

 Fragment:  [20, 25]
 Partition   40 orbitals into a total of    2 orbitals
            Index   Sing. Val. Space       
                0   0.83727198            0*
                1   0.81939117            0*
                2   0.15674432            2
                3   0.14914947            2
                4   0.02013429            1

 Fragment:  [30]
 Partition   40 orbitals into a total of    1 orbitals
            Index   Sing. Val. Space       
             

# Make Integrals

In [7]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(40, 1)
(40, 7)
(40, 40)


In [8]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [9]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

OSError: Unable to open file (file signature not found)

In [ ]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [ ]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)

In [ ]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)



WARN: orbitals [3 4] not symmetrized, norm = [0.76812236 0.76812236]



In [ ]:
print(Cdocc.shape)
print(Csing.shape)
P = 2*Cdocc@Cdocc.T + Csing@Csing.T
print(np.trace(P@S))
print(Ctot.shape)


(30, 4)
(30, 0)
8.0
(30, 4)


In [ ]:
Ptot = Ctot @ Ctot.T
Pact = Cact @ Cact.T 
Penv = Cenv @ Cenv.T
Psing = Csing @ Csing.T
Pdocc = Cdocc @ Cdocc.T
Pvirt = Cvirt @ Cvirt.T
print(Cenv.shape)

(30, 3)
